In [13]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)
!pip install transformers

import torch
import sys
import transformers
sys.path.append('/gdrive/My Drive/my_transformers')
sys.path.append('/gdrive/My Drive')
from my_transformers import(
    ElectraConfig,
    ElectraForSequenceClassification,
    ElectraTokenizer,
)


labels = ['0', '1']

config = ElectraConfig.from_pretrained(
            "monologg/koelectra-base-discriminator",
            num_labels=2,
            id2label={str(i): label for i, label in enumerate(labels)},
            label2id={label: i for i, label in enumerate(labels)},
        )

model = ElectraForSequenceClassification(config)
loaded = torch.load("/gdrive/My Drive/model_weight/pytorch_model.bin",map_location='cpu')  #torch.load(PATH,map_location), map_location은 model을 load하는 device 종류
model.load_state_dict(loaded,strict=False)

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator")

with open('/gdrive/My Drive/test_output.txt','w',encoding='utf-8') as fd_write:
  with open('/gdrive/My Drive/test_input.txt','r',encoding='utf-8') as fd_read:
    for sentence in fd_read.readlines():

      tokenized_sentence= tokenizer.tokenize(sentence)

      print('generated tokens: ',tokenized_sentence)

      gen_encoded = tokenizer.encode(tokenized_sentence, return_tensors="pt") # model내 vocabulary에 있는 id로 토큰을 mapping(encoding)

      model_input = gen_encoded

      classification_result = model(model_input)

      print(classification_result)

      result = torch.argmax(classification_result[0])
      if result:
        print(sentence,'= 긍정')
        fd_write.write((str(1)+'\n'))
      else:
        print(sentence,'= 부정')
        fd_write.write((str(0)+'\n'))

Mounted at /gdrive
generated tokens:  ['슬프', '##지만', '마음', '따뜻', '##해지는', '영화']
(tensor([[-2.0672,  2.3852]], grad_fn=<AddmmBackward>),)
슬프지만 마음 따뜻해지는 영화
 = 긍정
generated tokens:  ['음악', '연출', '스토리', '좋', '##았', '##음', '.', '.', '대만', '##영화', '중', '최고', '##인', '##듯']
(tensor([[-2.5859,  3.0954]], grad_fn=<AddmmBackward>),)
음악 연출 스토리 좋았음..대만영화 중 최고인듯
 = 긍정
generated tokens:  ['관객', '##을', '모욕', '##한', '영화']
(tensor([[ 3.2676, -3.1734]], grad_fn=<AddmmBackward>),)
관객을 모욕한 영화
 = 부정
generated tokens:  ['음악', '하', '##난', '죽여', '##주', '##네']
(tensor([[-1.6261,  1.9243]], grad_fn=<AddmmBackward>),)
음악 하난 죽여주네
 = 긍정
